In [56]:
import pandas as pd
import requests
from bs4 import BeautifulSoup,Comment
import re
import numpy as np
import urllib
import warnings
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)

### Function to retrieve stats
This is a function to retrieve both per game stats and advanced stats for an NBA regular season. The two arguments are first_year and last_year, with first_year being the first year that you want data from, and last_year being the last year you want data from. In this context, the year 2023 refers to the 2022-2023 season.

In addition to retrieving the stats, the function also adds in a 'Player Link' for each entry in the database. The player link is used in the salary functions below, to retrieve the salaries for every player in the database.

In [5]:
def get_stats(first_year, last_year):
    all_data = pd.DataFrame()
    #looping through years to get the data
    for year in range(first_year, last_year + 1):
        url_per_game = f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html#per_game_stats"
        try: 
            response = requests.get(url_per_game)
            response.raise_for_status()
        except Exception as err:
            print(f'An {err} error occured. Consider slowing down requests')
            return None
        soup = BeautifulSoup(response.text, 'html.parser')
        #This retrieves the unique player links
        player_links = [a['href'] for a in soup.select('td[data-stat="player"] > a')]
        tables_per_game = pd.read_html(url_per_game)
        df_per_game = tables_per_game[0]
        #This deletes unecessary 'filler' rows.
        df_per_game = df_per_game[df_per_game['Player'] != 'Player'].copy() 
        #Adding in the year, so that we can identify which season it was
        df_per_game.loc[:, 'Year'] = year
        df_per_game.loc[:, 'Player Link'] = player_links 
        #This is a meaningless column, so dropping
        df_per_game.drop(columns='Rk', inplace=True)
        #Getting the advanced data now
        url_advanced = f'https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html#advanced_stats'
        tables_advanced = pd.read_html(url_advanced)
        df_advanced = tables_advanced[0]
        df_advanced.loc[:, 'Year'] = year
        #These were all in df_per_game, so they are redundant
        df_advanced.drop(columns=['Rk', 'Pos', 'Age', 'G', 'MP'], inplace=True)
        df = pd.merge(df_per_game, df_advanced, how='inner', on=['Player', 'Tm', 'Year'])
        
        #If a player is traded, they return multiple entries in the dataframe. The first entry is the stats they had for the whole season
        # This is the one that we want to keep. keep = 'first' achieves this.
        df.drop_duplicates(subset=['Player', 'Year', 'Player Link'], keep='first', inplace=True)
        all_data = pd.concat([all_data, df])

    all_data.reset_index(drop=True, inplace=True)
    all_data.set_index('Player', inplace=True)

    return all_data


In [6]:
stats = get_stats(2010,2023)

In [7]:
stats.shape

(7060, 52)

In [8]:
stats.to_csv('../data/stats.csv')

### Function to retrieve salaries
This will retrieve a players career earnings. This code is adapted from https://stackoverflow.com/questions/66444486/scrapy-unable-to-locate-table-or-scrape-data-in-table

In [36]:
def get_player_salary(player_link):
    url = 'https://www.basketball-reference.com'
    full_url = url + player_link
    response = requests.get(full_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    tables = []
    for each in comments:
        if 'table' in str(each):
            try:
                tables.append(pd.read_html(str(each), attrs = {'id': 'all_salaries'})[0])
                break
            except:
                continue
    return tables[0]

In [98]:
#Testing function
print(get_player_salary('/players/a/allenra02.html'))

     Season                 Team   Lg        Salary
0   1996-97      Milwaukee Bucks  NBA    $1,785,000
1   1997-98      Milwaukee Bucks  NBA    $2,052,360
2   1998-99      Milwaukee Bucks  NBA    $2,320,000
3   1999-00      Milwaukee Bucks  NBA    $9,000,000
4   2000-01      Milwaukee Bucks  NBA   $10,130,000
5   2001-02      Milwaukee Bucks  NBA   $11,250,000
6   2002-03      Milwaukee Bucks  NBA   $12,375,000
7   2003-04  Seattle SuperSonics  NBA   $13,500,000
8   2004-05  Seattle SuperSonics  NBA   $14,625,000
9   2005-06  Seattle SuperSonics  NBA   $13,223,140
10  2006-07  Seattle SuperSonics  NBA   $14,611,570
11  2007-08       Boston Celtics  NBA   $16,000,000
12  2008-09       Boston Celtics  NBA   $18,388,430
13  2009-10       Boston Celtics  NBA   $18,776,860
14  2010-11       Boston Celtics  NBA   $10,000,000
15  2011-12       Boston Celtics  NBA   $10,000,000
16  2012-13           Miami Heat  NBA    $3,090,000
17  2013-14           Miami Heat  NBA    $3,229,050
18   Career 

### Function to retrieve all the salaries of players in a dataframe
Note: This takes awhile to run

In [87]:
def get_all_salaries(df):
    # Getting only unique player links, to avoid unecessary duplicates
    unique_player_links = df['Player Link'].unique()
    salary_data_frames = []
    for player_link in unique_player_links:
        try:
            salary_data = get_player_salary(player_link)
            salary_data['Player Link'] = player_link
            salary_data_frames.append(salary_data)
        except IndexError as ie:
            # Handle IndexErrors specifically
            print(f'Index Error occurred for player link {player_link}: {ie}')
        except requests.exceptions.RequestException as re:
            # Handle request exceptions specifically
            print(f'Request Error occurred for player link {player_link}: {re}')
            time.sleep(60)  # If there's a request error, sleep for 60 seconds before continuing
        except Exception as e:
            # Catch other general exceptions
            print(f'An unexpected error occurred for player link {player_link}: {e}')
        salary_data['Player Link'] = player_link
        salary_data_frames.append(salary_data)
    salary_data_df = pd.concat(salary_data_frames, ignore_index = True)
    return salary_data_df

In [100]:
#Testing get_all_salaries
get_all_salaries(df.head())

,Season,Team,Lg,Salary,Player Link
0,2007-08,Detroit Pistons,NBA,"$944,520",/players/a/afflaar01.html
1,2008-09,Detroit Pistons,NBA,"$1,015,440",/players/a/afflaar01.html
2,2009-10,Denver Nuggets,NBA,"$1,086,240",/players/a/afflaar01.html
3,2010-11,Denver Nuggets,NBA,"$1,959,577",/players/a/afflaar01.html
4,2011-12,Denver Nuggets,NBA,"$7,562,500",/players/a/afflaar01.html
...,...,...,...,...,...
105,2007-08,Dallas Mavericks,NBA,"$998,398",/players/a/allenma01.html
106,2008-09,Milwaukee Bucks,NBA,"$1,300,000",/players/a/allenma01.html
107,2009-10,Denver Nuggets,NBA,"$1,300,000",/players/a/allenma01.html
108,2010-11,Orlando Magic,NBA,"$1,229,255",/players/a/allenma01.html


In [88]:
salaries = get_all_salaries(stats)

Index Error occurred for player link /players/a/anderan02.html: list index out of range
Index Error occurred for player link /players/j/jeffeot01.html: list index out of range
Index Error occurred for player link /players/l/lafayol01.html: list index out of range
Index Error occurred for player link /players/c/cousima01.html: list index out of range
Index Error occurred for player link /players/e/ewingpa02.html: list index out of range
Index Error occurred for player link /players/b/byarsde01.html: list index out of range
Index Error occurred for player link /players/d/dentmju01.html: list index out of range
Index Error occurred for player link /players/d/dysonje01.html: list index out of range
Index Error occurred for player link /players/s/smithje01.html: list index out of range
Index Error occurred for player link /players/u/ubileed01.html: list index out of range
Index Error occurred for player link /players/w/wrighch02.html: list index out of range
Index Error occurred for player 

In [102]:
#index error occurs when a player doesnt have a salary table on their website

In [103]:
salaries.to_csv('../data/salaries.csv',index = False)

In [89]:
salaries

,Season,Team,Lg,Salary,Player Link
0,2007-08,Detroit Pistons,NBA,"$944,520",/players/a/afflaar01.html
1,2008-09,Detroit Pistons,NBA,"$1,015,440",/players/a/afflaar01.html
2,2009-10,Denver Nuggets,NBA,"$1,086,240",/players/a/afflaar01.html
3,2010-11,Denver Nuggets,NBA,"$1,959,577",/players/a/afflaar01.html
4,2011-12,Denver Nuggets,NBA,"$7,562,500",/players/a/afflaar01.html
...,...,...,...,...,...
22386,Career,(may be incomplete),NaN,"$1,062,303",/players/w/willivi01.html
22387,2021-22,Los Angeles Clippers,NBA,"$1,062,303",/players/w/willivi01.html
22388,Career,(may be incomplete),NaN,"$1,062,303",/players/w/willivi01.html
22389,2021-22,Los Angeles Clippers,NBA,"$1,062,303",/players/w/willivi01.html


### Getting 2023 Salaries
the 2022-2023 season isn't included in the previous data, so i will have to grab it from a different source. The team's current contracts. On a players basketball reference page they dont have the current contracts, but on a teams contracts page they don't have old contracts. So i had to collect the data from different sources.

 Note: this function will be outdated as soon as basketball reference updates this page. But it was the only way to get 2022-2023 salaries at the moment. 

In [96]:
def get_team_salary(team_name):
    url = f'https://www.basketball-reference.com/contracts/{team_name}.html'
    try:
        tables = pd.read_html(url,header = 1)
        df = tables[0]
        #indicating year, will be relevant when merging
        df['Year']= 2023
        # Removing filler rows
        df = df[df['Player']!='Team Totals']
        # The dataset has salaries for future years, but this is the only year we need.
        df=df.rename(columns = {'2022-23':'Salary'})
        df = df[['Player','Year','Salary']]
        df['Tm']= team_name
        return df
    except Exception as e:
        print(f"An error occurred, perhaps a basketball reference update: {e}")
        return None

In [97]:
#testing
get_team_salary('LAC')

,Player,Year,Salary,Tm
0,Paul George,2023,"$42,492,492",LAC
1,Kawhi Leonard,2023,"$42,492,492",LAC
2,Eric Gordon,2023,"$19,568,360",LAC
3,Norman Powell,2023,"$16,758,621",LAC
4,Marcus Morris,2023,"$16,372,093",LAC
5,Robert Covington,2023,"$12,307,692",LAC
6,Nicolas Batum,2023,"$10,843,350",LAC
7,Ivica Zubac,2023,"$10,123,457",LAC
8,Mason Plumlee,2023,"$9,080,417",LAC
9,Amir Coffey,2023,"$3,395,062",LAC


List of Team Abbreviations

In [98]:
team_names = [ 'LAL', 'NOP', 'NYK', 'HOU', 'CLE', 'CHI', 'GSW', 'DEN', 'BOS', 'OKC', 'CHO', 'MEM', 'ATL', 'MIA', 'MIN', 'ORL', 'IND', 'PHO', 'TOR', 'SAC', 'PHI', 'MIL', 'LAC', 'DET', 'SAS', 'UTA', 'DAL', 'WAS', 'POR', 'BRK']

In [99]:
len(team_names) 
#there are 30 NBA teams

30

Getting Current Salaries for all NBA teams

In [100]:
salary_2023 = pd.DataFrame()
for i in team_names:
    team_salary = get_team_salary(i)
    salary_2023 = pd.concat([salary_2023,team_salary],ignore_index = True)
    

In [101]:
salary_2023.to_csv('../data/salary_2023.csv',index = False)

### Retrieving Draft Results

In [102]:
def get_draft_results(first_year,last_year):
    all_data = pd.DataFrame()

    for year in range(first_year, last_year + 1):
        url_draft = f'https://www.basketball-reference.com/draft/NBA_{year}.html#stats'
        try:
            response = requests.get(url_draft)
            soup = BeautifulSoup(response.text, 'html.parser')
            player_links = [a['href'] for a in soup.select('td[data-stat="player"]>a')]
            tables_draft = pd.read_html(url_draft, header=1)
            df_draft = tables_draft[0]
            df_draft['Year Drafted'] = year
            df_draft= df_draft[df_draft['Rk'].isna()== False]
            df_draft = df_draft[df_draft['Rk']!='Rk']
            df_draft['Player Link']=player_links
            df_draft = df_draft[['Year Drafted', 'Player', 'Rk','Player Link']]
            all_data = pd.concat([all_data, df_draft])
        except urllib.error.HTTPError as e:
            print(f"HTTPError for year {year}: {e}")
        except Exception as e:
            print(f"An error occurred for year {year}: {e}")

    return all_data

In [103]:
draft =get_draft_results(2000,2022)

In [104]:
draft.head(50)

,Year Drafted,Player,Rk,Player Link
0,2000,Kenyon Martin,1,/players/m/martike01.html
1,2000,Stromile Swift,2,/players/s/swiftst01.html
2,2000,Darius Miles,3,/players/m/milesda01.html
3,2000,Marcus Fizer,4,/players/f/fizerma01.html
4,2000,Mike Miller,5,/players/m/millemi01.html
5,2000,DerMarr Johnson,6,/players/j/johnsde03.html
6,2000,Chris Mihm,7,/players/m/mihmch01.html
7,2000,Jamal Crawford,8,/players/c/crawfja01.html
8,2000,Joel Przybilla,9,/players/p/przybjo01.html
9,2000,Keyon Dooling,10,/players/d/doolike01.html


In [105]:
draft.to_csv('../data/draft.csv', index = False)

### Salary Cap Data

In [106]:

salarycap_url = 'https://www.spotrac.com/nba/cba/'
response = requests.get(salarycap_url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    salarycap = pd.read_html(str(table))[0]
else:
    print('Webpage not retrieved')
# In this context, year 2023 refers to 2023-2024, so i am fixing it to be in the same format as used previously
salarycap['Year']= salarycap['Year'] +1

In [107]:
salarycap = salarycap[['Year','Cap Maximum']]

In [108]:
salarycap.head()

,Year,Cap Maximum
0,2024,"$134,000,000"
1,2023,"$123,655,000"
2,2022,"$112,414,000"
3,2021,"$109,140,000"
4,2020,"$109,140,000"


In [109]:
salarycap.to_csv('../data/salarycap.csv',index = False)

### Merging Dataframes and Cleaning

In [404]:
salary = pd.read_csv('../data/salaries.csv')
stats = pd.read_csv('../data/stats.csv')
draft = pd.read_csv('../data/draft.csv')
salarycap = pd.read_csv('../data/salarycap.csv')
salary_2023 = pd.read_csv('../data/salary_2023.csv')

### Cleaning Stats

In [405]:
stats.shape

(7060, 53)

In [406]:
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Player Link,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,0.465,1.3,3.0,0.434,2.0,4.1,0.488,0.557,0.9,1.2,0.735,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010,/players/a/afflaar01.html,10.9,0.576,0.426,0.168,3.1,9.9,6.5,9.3,1.0,1.0,10.5,14.0,NaN,2.8,1.4,4.3,0.092,NaN,-0.2,-0.2,-0.4,0.9
1,Alexis Ajinça,C,21,CHA,6,0,5.0,0.8,1.7,0.500,0.0,0.0,NaN,0.8,1.7,0.500,0.500,0.0,0.2,0.000,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010,/players/a/ajincal01.html,6.3,0.479,0.000,0.100,4.1,11.8,8.0,0.0,1.8,2.7,16.1,19.3,NaN,-0.1,0.0,0.0,-0.013,NaN,-6.3,1.0,-5.3,0.0
2,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,0.495,0.1,0.2,0.313,7.4,14.8,0.498,0.497,2.9,3.9,0.757,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010,/players/a/aldrila01.html,18.2,0.535,0.014,0.260,8.1,18.6,13.3,9.9,1.3,1.3,7.4,22.9,NaN,5.5,3.3,8.8,0.145,NaN,1.4,-0.2,1.2,2.3
3,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,0.167,0.0,0.1,0.000,0.1,0.6,0.200,0.167,0.3,0.4,0.667,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010,/players/a/alexajo01.html,2.8,0.273,0.167,0.500,7.8,11.3,9.6,9.3,1.8,2.6,0.0,11.3,NaN,0.0,0.0,0.0,0.030,NaN,-9.1,0.9,-8.3,0.0
4,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,0.397,0.0,0.1,0.167,0.9,2.2,0.409,0.401,0.2,0.3,0.923,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010,/players/a/allenma01.html,5.9,0.431,0.052,0.112,9.2,11.5,10.4,5.1,1.2,0.8,15.3,14.0,NaN,-0.3,0.3,0.1,0.009,NaN,-4.7,-1.0,-5.7,-0.4


In [407]:
stats.drop(columns = ['Unnamed: 19', 'Unnamed: 24'],inplace = True)

In [408]:
stats.isnull().sum()

Player           0
Pos              0
Age              0
Tm               0
G                0
GS               0
MP               0
FG               0
FGA              0
FG%             34
3P               0
3PA              0
3P%            679
2P               0
2PA              0
2P%             72
eFG%            34
FT               0
FTA              0
FT%            311
ORB              0
DRB              0
TRB              0
AST              0
STL              0
BLK              0
TOV              0
PF               0
PTS              0
Year             0
Player Link      0
PER              2
TS%             33
3PAr            34
FTr             34
ORB%             2
DRB%             2
TRB%             2
AST%             2
STL%             2
BLK%             2
TOV%            29
USG%             2
OWS              0
DWS              0
WS               0
WS/48            2
OBPM             0
DBPM             0
BPM              0
VORP             0
dtype: int64

In [409]:
stats.dtypes

Player          object
Pos             object
Age              int64
Tm              object
G                int64
GS               int64
MP             float64
FG             float64
FGA            float64
FG%            float64
3P             float64
3PA            float64
3P%            float64
2P             float64
2PA            float64
2P%            float64
eFG%           float64
FT             float64
FTA            float64
FT%            float64
ORB            float64
DRB            float64
TRB            float64
AST            float64
STL            float64
BLK            float64
TOV            float64
PF             float64
PTS            float64
Year             int64
Player Link     object
PER            float64
TS%            float64
3PAr           float64
FTr            float64
ORB%           float64
DRB%           float64
TRB%           float64
AST%           float64
STL%           float64
BLK%           float64
TOV%           float64
USG%           float64
OWS        

In [410]:
stats.duplicated().sum()

0

### Cleaning Salary

In [411]:
salary.head()

,Season,Team,Lg,Salary,Player Link
0,2007-08,Detroit Pistons,NBA,"$944,520",/players/a/afflaar01.html
1,2008-09,Detroit Pistons,NBA,"$1,015,440",/players/a/afflaar01.html
2,2009-10,Denver Nuggets,NBA,"$1,086,240",/players/a/afflaar01.html
3,2010-11,Denver Nuggets,NBA,"$1,959,577",/players/a/afflaar01.html
4,2011-12,Denver Nuggets,NBA,"$7,562,500",/players/a/afflaar01.html


In [412]:
salary['Season'].value_counts()

Career     3213
2021-22    1570
2020-21    1292
2017-18    1287
2018-19    1241
2019-20    1230
2016-17    1191
2014-15    1119
2015-16    1103
2012-13     999
2011-12     944
2010-11     936
2009-10     919
2013-14     863
2008-09     780
2007-08     697
2006-07     623
2005-06     525
2004-05     435
2003-04     354
2002-03     286
2001-02     234
2000-01     168
1999-00     140
1998-99      90
1997-98      64
1996-97      46
1995-96      26
1994-95      10
1993-94       4
1992-93       2
Name: Season, dtype: int64

In [413]:
#dropping Career as this does not matter for the model (and it would be data leakage to include)
salary = salary[salary['Season']!='Career']

In [414]:
#now i will convert the season column to the desired format (year)
def extract_year(season):
    last_two_chars = season.split('-')[1]
    first_four_chars = season.split('-')[0]
    if int(first_four_chars)>=1999:
        return int('20'+last_two_chars)
    else:
        return int('19'+last_two_chars)
        

In [415]:
#testing
extract_year('1999-00')

2000

In [416]:
salary['Year'] = salary['Season'].apply(extract_year)

In [417]:
salary['Year'].value_counts()

2022    1570
2021    1292
2018    1287
2019    1241
2020    1230
2017    1191
2015    1119
2016    1103
2013     999
2012     944
2011     936
2010     919
2014     863
2009     780
2008     697
2007     623
2006     525
2005     435
2004     354
2003     286
2002     234
2001     168
2000     140
1999      90
1998      64
1997      46
1996      26
1995      10
1994       4
1993       2
Name: Year, dtype: int64

In [418]:
salary.drop_duplicates(subset=['Player Link','Year'],inplace = True)

In [419]:
salary = salary[['Year','Player Link','Salary']]

In [420]:
salary.isnull().sum()

Year           0
Player Link    0
Salary         1
dtype: int64

In [421]:
salary[salary['Salary'].isnull()]

,Year,Player Link,Salary
14106,2015,/players/s/shengto01.html,NaN


In [422]:
#checked and this player has an empty salary, and recorded no stats that season. So i will delete him from the df.
salary.dropna(inplace =True)

In [423]:
salary['Salary'].value_counts()

< $Minimum     247
$4,000,000      79
$3,000,000      79
$5,000,000      68
$473,604        55
              ... 
$1,359,360       1
$1,563,000       1
$19,014,187      1
$20,370,437      1
$1,062,303       1
Name: Salary, Length: 4958, dtype: int64

In [424]:
salary[salary['Salary'] =='< $Minimum']

,Year,Player Link,Salary
14405,2018,/players/b/brownlo01.html,< $Minimum
15032,2021,/players/n/nunnaja01.html,< $Minimum
15274,2018,/players/w/woltena01.html,< $Minimum
15384,2018,/players/b/brownma02.html,< $Minimum
15449,2018,/players/c/cooleja01.html,< $Minimum
...,...,...,...
22046,2022,/players/s/smartja01.html,< $Minimum
22112,2022,/players/w/waterli01.html,< $Minimum
22128,2022,/players/w/wiggili01.html,< $Minimum
22140,2022,/players/w/wrighmc01.html,< $Minimum


In [425]:
#Hard to identify what this means. From inspection, it looks like these players are making 100k or less in other years. Im going to make them 50k, as a closest guess.

condition = salary['Salary'] == '< $Minimum'
salary.loc[condition, 'Salary'] = '50,000'

In [426]:
salary.isnull().sum()

Year           0
Player Link    0
Salary         0
dtype: int64

In [427]:
def clean_salary_column(salary_column):
    return salary_column.apply(lambda x: int(re.sub("[^0-9]", "", x)) if pd.notna(x) else np.nan)

In [428]:
salary['Salary']=clean_salary_column(salary['Salary'])

### Merging Salary and Stats

Merging on Player Link and Year. Two players may have the same name, but not the same player link.

In [430]:
df = pd.merge(stats,salary, how = 'left', on = ['Player Link','Year'])

In [431]:
df.shape

(7060, 52)

### Cleaning Salary 2023

In [433]:
salary_2023[salary_2023.duplicated(subset='Player', keep = False)]

,Player,Year,Salary,Tm
17,NaN,2023,NaN,LAL
49,Isaiah Roby,2023,NaN,NYK
70,NaN,2023,NaN,HOU
73,Danny Green,2023,"$9,710,528",HOU
74,Justin Holiday,2023,"$6,292,440",HOU
76,Ty Jerome,2023,"$4,220,057",HOU
77,Théo Maledon,2023,"$1,900,000",HOU
88,Danny Green,2023,"$2,000,000",CLE
93,Sam Merrill,2023,"$850,000",CLE
96,NaN,2023,NaN,CLE


There are duplicates. If a player played for multiple teams, his name will occur multiple times. The players salaries should be the sum of both of their salaries that season.

In [434]:
salary_2023['Salary'] = clean_salary_column(salary_2023['Salary'])

In [435]:
grouped_salary_2023 = salary_2023.groupby(['Player'], as_index = False)['Salary'].sum()

In [436]:
#Testing that it worked

In [437]:
grouped_salary_2023.duplicated(subset = 'Player').sum()

0

In [438]:
grouped_salary_2023[grouped_salary_2023['Player']== 'Russell Westbrook']

,Player,Salary
455,Russell Westbrook,47559433.0


In [439]:
salary_2023[salary_2023['Player']== 'Russell Westbrook']

,Player,Year,Salary,Tm
445,Russell Westbrook,2023,495955.0,LAC
515,Russell Westbrook,2023,47063478.0,UTA


In [440]:
#it worked!

In [441]:
grouped_salary_2023['Year']= 2023

### Merging salary_2023

In [442]:
df_2 = pd.merge(df,grouped_salary_2023, how = 'left',on =  ['Player', 'Year'])

In [443]:
df_2['Salary_x']=df_2['Salary_x'].combine_first(df_2['Salary_y'])

In [444]:
#Testing to see this worked correctly the count where they are the same should be the count where salary_y is not null
print((df_2['Salary_y']==df_2['Salary_x']).sum())
print(df_2['Salary_y'].notna().sum())

521
521


In [445]:
df_2.drop(columns = 'Salary_y',inplace = True)
df_2.rename(columns = {'Salary_x': 'Salary'},inplace = True)

### Cleaning and Merging Draft

In [446]:
draft.duplicated().sum()

0

In [447]:
draft = draft[['Year Drafted','Rk','Player Link']]

In [448]:
df3 = pd.merge(df_2,draft,on = 'Player Link', how = 'left' )

In [449]:
salarycap.head()

,Year,Cap Maximum
0,2024,"$134,000,000"
1,2023,"$123,655,000"
2,2022,"$112,414,000"
3,2021,"$109,140,000"
4,2020,"$109,140,000"


In [450]:
salarycap['Cap Maximum']= clean_salary_column(salarycap['Cap Maximum'])

In [451]:
df_merged = pd.merge(df3, salarycap, on ='Year', how = 'left')

In [452]:
df_merged

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Player Link,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Salary,Year Drafted,Rk,Cap Maximum
0,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,0.465,1.3,3.0,0.434,2.0,4.1,0.488,0.557,0.9,1.2,0.735,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010,/players/a/afflaar01.html,10.9,0.576,0.426,0.168,3.1,9.9,6.5,9.3,1.0,1.0,10.5,14.0,2.8,1.4,4.3,0.092,-0.2,-0.2,-0.4,0.9,1086240.0,2007.0,27.0,57700000
1,Alexis Ajinça,C,21,CHA,6,0,5.0,0.8,1.7,0.500,0.0,0.0,NaN,0.8,1.7,0.500,0.500,0.0,0.2,0.000,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010,/players/a/ajincal01.html,6.3,0.479,0.000,0.100,4.1,11.8,8.0,0.0,1.8,2.7,16.1,19.3,-0.1,0.0,0.0,-0.013,-6.3,1.0,-5.3,0.0,1372080.0,2008.0,20.0,57700000
2,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,0.495,0.1,0.2,0.313,7.4,14.8,0.498,0.497,2.9,3.9,0.757,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010,/players/a/aldrila01.html,18.2,0.535,0.014,0.260,8.1,18.6,13.3,9.9,1.3,1.3,7.4,22.9,5.5,3.3,8.8,0.145,1.4,-0.2,1.2,2.3,5844827.0,2006.0,2.0,57700000
3,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,0.167,0.0,0.1,0.000,0.1,0.6,0.200,0.167,0.3,0.4,0.667,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010,/players/a/alexajo01.html,2.8,0.273,0.167,0.500,7.8,11.3,9.6,9.3,1.8,2.6,0.0,11.3,0.0,0.0,0.0,0.030,-9.1,0.9,-8.3,0.0,2583360.0,2008.0,8.0,57700000
4,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,0.397,0.0,0.1,0.167,0.9,2.2,0.409,0.401,0.2,0.3,0.923,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010,/players/a/allenma01.html,5.9,0.431,0.052,0.112,9.2,11.5,10.4,5.1,1.2,0.8,15.3,14.0,-0.3,0.3,0.1,0.009,-4.7,-1.0,-5.7,-0.4,1300000.0,NaN,NaN,57700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7055,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2023,/players/y/youngth01.html,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,8000000.0,2007.0,12.0,123655000
7056,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2023,/players/y/youngtr01.html,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,37096500.0,2018.0,5.0,123655000
7057,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2023,/players/y/yurtsom01.html,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,1752638.0,NaN,NaN,123655000
7058,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2023,/players/z/zelleco01.html,16.4,0.659,0.034,0.593,13.0,21.8,17.3,7.2,0.7,1.9,15.8,18.1,0.4,0.3,0.7,0.147,-2.0,-0.7,-2.8,0.0,517060.0,2013.0,4.0,123655000


In [453]:
df_merged.isnull().sum()

Player             0
Pos                0
Age                0
Tm                 0
G                  0
GS                 0
MP                 0
FG                 0
FGA                0
FG%               34
3P                 0
3PA                0
3P%              679
2P                 0
2PA                0
2P%               72
eFG%              34
FT                 0
FTA                0
FT%              311
ORB                0
DRB                0
TRB                0
AST                0
STL                0
BLK                0
TOV                0
PF                 0
PTS                0
Year               0
Player Link        0
PER                2
TS%               33
3PAr              34
FTr               34
ORB%               2
DRB%               2
TRB%               2
AST%               2
STL%               2
BLK%               2
TOV%              29
USG%               2
OWS                0
DWS                0
WS                 0
WS/48              2
OBPM         

In [454]:
df_merged.to_csv('../data/dataset_eda.csv',index = False)